<a href="https://colab.research.google.com/github/renatabmagro/Simplex_implementation/blob/main/Simplex_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementação do Método Simplex**

> Atividade extra Otimização Linear I 

> Aluna: Renata Bulling Magro


**Método Big M - Tableau**

- O script resolve problemas de PL de maximização e minimização; considera restrições '>=' e '<=' 

**Descrição geral:**

- gen_matrix() produz uma matriz para receber restrições e a função objetivo (maximizar ou minimizar)
  - Ela recebe var (número de variáveis de decisão) e cons (número de restrições do problema) como parâmetros.
  - Por exemplo, gen_matrix(2,3) criará uma matriz 4x7 (para a forma padrão)

- constrain() restringe o problema. Essa função o problema como o primeiro argumento e uma string como o segundo argumento.
  - A cadeia de caracteres deve ser inserida na forma de '1,2,G,10' -> o que significa que 1(x1) + 2(x2) >= 10.
  - É possível utilizar 'L' para '<=' em vez de 'G'. 

- obj() corresponde à definição da função objetivo. Os parâmetros devem ser inseridos após as restrições, na forma de '1,2,0' -> o que significa 1(x1) +2(x2) +0
  - O termo final é sempre reservado para uma constante e 0 não pode ser omitido.
  - maxz() para resolver um problema de maximização. 
  - minz() para resolver um problema de minimização.





In [ ]:
import numpy as np

# cria uma matriz vazia com o tamanho adequado de variáveis e restrições.
def gen_matrix(var,cons):
    tab = np.zeros((cons+1, var+cons+2))
    return tab

# verificação pivô: verifica a coluna mais à direita acima da última linha. Se existirem valores negativos, outro pivô é necessário. 
def next_round_r(table):
    m = min(table[:-1,-1])
    if m>= 0:
        return False
    else:
        return True

# verificação pivô: verifica a linha inferior, sem considerar a coluna final, buscando valores negativos. Se existirem valores negativos, outro pivô é necessário. 
def next_round(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m>=0:
        return False
    else:
        return True

# semelhante à função 'next_round_r', mas retorna o índice da linha do elemento negativo na coluna mais à direita
def find_neg_r(table):
    # lc = número de colunas; lr = número de linhas.
    lc = len(table[0,:])
    # avalia cada linha na última coluna para identificar o menor valor (sem considerar a última)
    m = min(table[:-1,lc-1])
    if m<=0:
        # n = índice da linha com o menor valor
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

# retorna o índice da coluna com o elemento negativo na linha inferior
def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        # n = índice da linha para m 
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

# localiza o elemento pivô na tabela para remover o elemento negativo da coluna mais à direita
def loc_piv_r(table):
        total = []
        # r = índice da linha do elemento negativo
        r = find_neg_r(table)
        # encontra todos os elementos na linha, r, excluindo a última coluna
        row = table[r,:-1]
        # encontra o menor valor na linha, excluindo a última coluna
        m = min(row)
        # c = índice da coluna para que vai entrar 
        c = np.where(row == m)[0][0]
        # todos os elementos da coluna
        col = table[:-1,c]
        # percorre a coluna para encontrar a menor razão positiva
        for i, b in zip(col,table[:-1,-1]):
            # i não pode ser zero e b/i deve ser positivo
            if i**2>0 and b/i>0:
                total.append(b/i)
            else:
                # para os elementos que não satisfazem as condições acima
                total.append(0)
        element = max(total)
        for t in total:
            if t > 0 and t < element:
                element = t
            else:
                continue

        index = total.index(element)
        return [index,c]

# processo semelhante, retorna um elemento de matriz específico para ser pivoteado
def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if i**2>0 and b/i>0:
                total.append(b/i)
            else:
                # para os elementos que não satisfazem as condições acima
                total.append(0)
        element = max(total)
        for t in total:
            if t > 0 and t < element:
                element = t
            else:
                continue

        index = total.index(element)
        return [index,n]

# Recebe uma string de entrada e retorna uma lista de números a serem organizados na tabela
def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

# A linha final da tabela em um problema mínimo é o oposto de um problema de maximização, portanto, os elementos são multiplicados por (-1)
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]
    return table

# gera as variáveis x1,x2,...xn para o número de variáveis inseridas
def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

# pivoteamento da tabela: os elementos negativos sejam removidos da última linha e da última coluna
def pivot(row,col,table):
    # número de linhas
    lr = len(table[:,0])
    # número de colunas
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0: 
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Esse elemento não pode ser pivoteado.')

# verifica se há espaço na matriz para adicionar outra restrição
def add_cons(table):
    lr = len(table[:,0])
    # verifica SE existem pelo menos 2 linhas com todos os elementos zero
    empty = []
    # iteração de cada linha
    for i in range(lr):
        total = 0
        for j in table[i,:]:
            # usar o valor ao quadrado para que (-x) e (+x) não se cancelem mutuamente
            total += j**2
        if total == 0:
            # acrescentar zero à lista apenas se todos os elementos em uma linha forem zero
            empty.append(total)
    # Há pelo menos duas linhas com todos os elementos zero se a condição for verdadeira: 
    if len(empty)>1:
        return True
    else:
        return False

# adiciona uma restrição à matriz
def constrain(table,eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1
        # configura o contador para iterar pelo comprimento total das linhas
        j = 0
        while j < lr:
            # iteração por linha
            row_check = table[j,:]
            # o total será a soma das entradas das linhas
            total = 0
            # encontra a primeira linha com todos os elementos zero
            for i in row_check:
                total += float(i**2)
            if total == 0:
                # encontramos a primeira linha com todas os elementos zero
                row = row_check
                break
            j +=1

        eq = convert(eq)
        i = 0
        # itera todos os termos das restrições, excluindo o último
        while i<len(eq)-1:
            # atribui os valores de linha de acordo com a equação
            row[i] = eq[i]
            i +=1
        
        row[-1] = eq[-1]

        # adiciona variáveis de folga de acordo com a localização na tabela
        row[var+j] = 1
    else:
        print('Não é possível adicionar outra restrição.')

# verificação para determinar se uma função objetivo pode ser adicionada à tabela
def add_obj(table):
    lr = len(table[:,0])
    # verifica se existe uma linha com todos os elementos zero
    empty = []
    # itera todos os elementos em cada linha
    for i in range(lr):
        total = 0
        for j in table[i,:]:
            # utiliza o valor ao quadrado para que (-x) e (+x) não se cancelem mutuamente
            total += j**2
        if total == 0:
            # acrescentar zero à lista apenas se todos os elementos na linha são iguais a zero
            empty.append(total)
    # Existe uma linha com todos os elementos zero se a condição for verdadeira:
    if len(empty)==1:
        return True
    else:
        return False

# Adiciona a função objetivo na tabela
def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0
    # itera por todos os termos da função de restrição, excluindo o último
        while i<len(eq)-1:
            # atribui valores de linha de acordo com a equação
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('Você deve terminar de adicionar as restrições antes que a função objetiva possa ser adicionada.')

# resolve o problema de maximização para obter a solução ideal; retorna um dicionário com as chaves x1,x2...xn e max.
def maxz(table, output='summary'):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)

    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    for k,v in val.items():
        val[k] = round(v,6)
    if output == 'table':
        return table
    else:
        return val

# resolve o problema de minimização para obter a solução ideal; retorna um dicionário com as chaves x1,x2...xn e min.
def minz(table, output='summary'):
    table = convert_min(table)

    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)

    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['min'] = table[-1,-1]*-1
    for k,v in val.items():
        val[k] = round(v,6)
    if output == 'table':
        return table
    else:
        return val



Resolvendo:

1) 
      
      max  5x1 + 10x2
   
  s.a. 
       
       2x1 - 1x2 >= 10
       
       1x1 + 1x2 <= 20
       
       x1, x2 >=0


2) 

    min 2x1 + 7x2

  s.a. 

        2x1 + 5x2 >= 30
        -3x1 + 5x2 >= 5
        8x1 + 3x2 <= 85
        -9x1 + 7x2 <= 42

In [ ]:
if __name__ == "__main__":

    m = gen_matrix(2,2) # (nº variáveis de decisão, nº restrições)
    constrain(m,'2,-1,G,10')
    constrain(m,'1,1,L,20')
    obj(m,'5,10,0')
    print(maxz(m)) # função de maximização

    m = gen_matrix(2,4) # (nº variáveis de decisão, nº restrições)
    constrain(m,'2,5,G,30')
    constrain(m,'-3,5,G,5')
    constrain(m,'8,3,L,85')
    constrain(m,'-9,7,L,42')
    obj(m,'2,7,0')
    print(minz(m)) # função de minimização

{'x1': 10.0, 'x2': 10.0, 'max': 150.0}
{'x1': 5.0, 'x2': 4.0, 'min': 38.0}
